In [ ]:
class FooBar:
    def __init__(self):
        return

    def set_up(self):
        return

    def run(self, query: str):
        return "My query is " + query

app = FooBar()
app.run(query="hello")

In [ ]:
!pip install --upgrade google-cloud-aiplatform==1.49.0 langchain==0.1.16 langchain-google-vertexai==1.0.2 pydantic

In [10]:
PROJECT_ID = "andrewcooley-test-project"
LOCATION = "us-central1"
STAGING_BUCKET = "gs://andrewcooley-reasoning-engine"

import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [2]:
import requests
from vertexai.preview import reasoning_engines

In [3]:
model = "gemini-1.5-pro-preview-0409"

In [4]:
def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date."""
    import requests

    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [5]:
get_exchange_rate(currency_from="USD", currency_to="SEK")

{'amount': 1.0, 'base': 'USD', 'date': '2024-04-22', 'rates': {'SEK': 10.9119}}

In [6]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
    agent_executor_kwargs={"return_intermediate_steps": True},
)

In [7]:
agent.query(input="What's the exchange rate from US dollars to Swedish currency today?")

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'Today, 1 US dollar is equal to 10.9119 Swedish krona. \n',
 'intermediate_steps': [(AgentActionMessageLog(tool='get_exchange_rate', tool_input={'currency_from': 'USD', 'currency_to': 'SEK'}, log="\nInvoking: `get_exchange_rate` with `{'currency_from': 'USD', 'currency_to': 'SEK'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_exchange_rate', 'arguments': '{"currency_from": "USD", "currency_to": "SEK"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [{'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability_label': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability_label': 'NEGLIGIBLE', 'blocked': 

In [8]:
agent = reasoning_engines.LangchainAgent(
    model=model,
    tools=[get_exchange_rate],
)

In [11]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        "google-cloud-aiplatform[reasoningengine,langchain]",
        "requests",
    ],
)

Creating bucket andrewcooley-reasoning-engine in location='us-central1'
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/reasoning_engine.pkl
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://andrewcooley-reasoning-engine/reasoning_engine/dependencies.tar.gz
failed to generate schema: issubclass() arg 1 must be a class
Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/619758184732/locations/us-central1/reasoningEngines/8102890523313831936/operations/3136289332625670144
ReasoningEngine created. Resource name: projects/619758184732/locations/us-central1/reasoningEngines/8102890523313831936
To use this ReasoningEngine in another session:
reasoning_engine = vertexai.preview.reasoning_engines.ReasoningEngine('projects/619758184732/locations/us-central1/reasoningEngines/8102890523313831936')


In [12]:
remote_agent.query(
    input="What's the exchange rate from US dollars to Swedish currency today?"
)

{'input': "What's the exchange rate from US dollars to Swedish currency today?",
 'output': 'Today, April 22, 2024, 1 US dollar is equal to 10.9119 Swedish kronor. \n'}

In [13]:
reasoning_engines.ReasoningEngine.list()

 resource name: projects/619758184732/locations/us-central1/reasoningEngines/8102890523313831936]